In [42]:
import numpy as np
import pandas as pd

In [43]:
df = pd.read_csv('Assignment-gene_data.csv')
df = pd.DataFrame(df)
df.head()

/home/iiserb/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning:

Columns (3,14) have mixed types. Specify dtype option on import or set low_memory=False.



,Unnamed: 0,symbol,S1,S2,S3,S4,S5,S6,S7,S8,...,S21,S22,S23,S24,S25,S26,S27,S28,S29,S30
0,1,Zzz3,253.401685,402.8508776,230.957463,688.486264,358.067836,344.962350,441.155001,301.149694,...,507.882774,465.081486,415.478767,297.061149,488.848114,367.912141,307.199908,347.802509,411.227261,283.709694
1,2,Zzef1,21.152797,21.15279694,21.152797,21.152797,21.152797,21.152797,21.152797,21.152797,...,21.152797,21.152797,21.152797,21.152797,21.152797,21.152797,21.152797,21.152797,21.152797,21.152797
2,3,Zyx,40.578958,49.23516006,59.472005,46.212152,61.969671,55.732843,53.819870,54.315691,...,33.512262,36.751853,36.952457,35.718610,27.346198,32.647887,42.006833,32.324596,41.426033,32.754971
3,4,Zyg11b,26.373102,35.00190057,32.389728,34.932372,35.192365,33.858459,27.948378,21.152797,...,21.152797,21.152797,21.152797,21.152797,21.152797,21.152797,21.152797,21.152797,21.152797,21.152797
4,5,Zyg11a,21.152797,21.15279694,21.152797,21.152797,21.152797,21.152797,21.152797,21.152797,...,21.152797,21.152797,21.152797,21.152797,21.152797,21.152797,21.152797,21.152797,21.152797,21.152797


In [44]:
df2 = pd.read_csv('Assignment-Meta_data sheet.csv')
df2 = pd.DataFrame(df2)
df2.head()

,sIdx,Time,Unit
0,S1,9,Hours
1,S2,9,Hours
2,S3,9,Hours
3,S4,7,Hours
4,S5,7,Hours


In [45]:
#X = df.iloc[0:,2:].values
y = df.iloc[:,1].values

In [46]:
df1 = df.where(df.applymap(lambda x: str(x).isalnum() is False))
df1 = df1.fillna(0.0)
X = df1.iloc[0:,2:].values
X = X.astype(float)

In [47]:
from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)

# 1 - Eigendecomposition - Computing Eigenvectors and Eigenvalues

## Covariance Matrix

In [48]:
import numpy as np
mean_vec = np.mean(X_std, axis=0)
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
#print('Covariance matrix \n%s' %cov_mat)

In [49]:
#print('NumPy covariance matrix: \n%s' %np.cov(X_std.T))


In [50]:
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)

#print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)


Eigenvalues 
[2.42964461e+01 2.23770201e+00 1.39246721e+00 9.95406663e-01
 3.15734421e-01 1.72182656e-01 1.20125032e-01 8.52449625e-02
 7.43657524e-02 5.18721014e-02 4.18005262e-02 3.43968982e-02
 2.85211048e-02 2.26709793e-02 1.79366743e-02 1.68621657e-02
 1.41674219e-02 1.16280791e-02 1.09234764e-02 9.47992499e-03
 2.20558968e-03 8.13304970e-03 7.19112488e-03 6.16404718e-03
 5.82498965e-03 5.60878604e-03 5.00486859e-03 3.31168887e-03
 3.77693598e-03 4.18340516e-03]


## Correlation Matrix

In [51]:
cor_mat1 = np.corrcoef(X_std.T)

eig_vals, eig_vecs = np.linalg.eig(cor_mat1)

#print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)


Eigenvalues 
[2.42953620e+01 2.23760216e+00 1.39240508e+00 9.95362247e-01
 3.15720332e-01 1.72174973e-01 1.20119672e-01 8.52411587e-02
 7.43624341e-02 5.18697868e-02 4.17986610e-02 3.43953633e-02
 2.85198322e-02 2.26699677e-02 1.79358739e-02 1.68614133e-02
 1.41667898e-02 1.16275602e-02 1.09229890e-02 9.47950198e-03
 2.20549126e-03 8.13268679e-03 7.19080400e-03 3.31154110e-03
 6.16377214e-03 5.82472974e-03 5.60853577e-03 5.00464526e-03
 3.77676745e-03 4.18321849e-03]


# 2 - Selecting Principal Components

Normalization of data

In [53]:
for ev in eig_vecs:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))


In [54]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0])

Eigenvalues in descending order:
24.295362015095662
2.2376021601312543
1.3924050757525022
0.9953622469557909
0.31572033249584947
0.17217497323310668
0.12011967225779117
0.08524115874677898
0.074362434118428
0.051869786822721684
0.041798660993265284
0.03439536333077761
0.02851983218507281
0.022669967683088363
0.017935873935823007
0.01686141331658112
0.014166789773240415
0.011627560214170822
0.010922988971147634
0.009479501981811711
0.008132686792803729
0.007190804002369863
0.006163772138175464
0.005824729737954111
0.00560853576878121
0.005004645263413443
0.004183218488559366
0.003776767453837126
0.003311541095890872
0.002205491263358622


In [55]:
import plotly as py
from plotly.graph_objs import *
import plotly.tools as tls


In [56]:
tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

trace1 = Bar(
        x=['PC %s' %i for i in range(1,30)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,30)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data = Data([trace1, trace2])

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Explained variance by different principal components')

fig = Figure(data=data, layout=layout)
#py.iplot(fig)
py.offline.plot( fig, filename='PCA_analysis' )

/home/iiserb/anaconda3/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.


/home/iiserb/anaconda3/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:558: DeprecationWarning:

plotly.graph_objs.YAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.YAxis
  - plotly.graph_objs.layout.scene.YAxis


/home/iiserb/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:608: UserWarning:

Your filename `PCA_analysis` didn't end with .html. Adding .html to the end of your file.



'file:///home/iiserb/Music/PCA_analysis.html'

In [57]:
matrix_w = np.hstack((eig_pairs[0][1].reshape(30,1), 
                      eig_pairs[1][1].reshape(30,1),
                     eig_pairs[2][1].reshape(30,1),
                     eig_pairs[3][1].reshape(30,1),
                     eig_pairs[4][1].reshape(30,1),
                     eig_pairs[5][1].reshape(30,1)))

print('Matrix W:\n', matrix_w)

Matrix W:
 [[ 1.58487523e-01 -3.65617206e-01  1.31431077e-01 -1.01566628e-01
  -3.16845125e-01  1.85136743e-01]
 [ 1.68892201e-01 -3.20783573e-01  3.18983758e-02 -1.80313316e-01
  -2.36185106e-01  1.20040503e-01]
 [ 1.72105681e-01 -2.87598341e-01  1.64427065e-02 -2.32105879e-01
  -1.86849676e-01  2.47887551e-01]
 [ 1.71062669e-01 -3.29003301e-01  2.60643472e-04 -1.64068656e-02
  -1.89676776e-01 -2.54997915e-01]
 [ 1.72522610e-01 -1.77579964e-01 -2.95276587e-01 -2.02933902e-01
   2.33858028e-01  3.73418213e-02]
 [ 1.78705943e-01 -2.04408284e-01 -2.12087918e-01 -1.39020116e-01
   2.68180174e-01 -1.53893163e-01]
 [ 1.81501032e-01 -1.43659503e-01 -2.89674150e-01  3.88612443e-02
   1.92984706e-01 -1.00074152e-03]
 [ 1.81478299e-01 -5.92876202e-02 -3.22161764e-01 -2.36996364e-04
   2.89103158e-01 -1.21815972e-01]
 [ 1.86338661e-01 -1.33560391e-01 -2.61481261e-01  6.68105755e-02
   1.20249343e-01 -1.20617024e-01]
 [ 1.84770910e-01  5.27684868e-02 -2.02213053e-01  2.98765568e-01
  -7.11897445e